# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("output_data/cities.csv")
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,upernavik,72.7868,-56.1549,250.51,69,28,2.21,GL,1612475094
1,bowen,-20.0167,148.2333,298.15,88,81,3.09,AU,1612475095
2,hermanus,-34.4187,19.2345,293.15,78,10,0.45,ZA,1612475095
3,saskylakh,71.9167,114.0833,232.23,75,0,1.71,RU,1612475095
4,bredasdorp,-34.5322,20.0403,293.15,77,98,2.57,ZA,1612475095
...,...,...,...,...,...,...,...,...,...
556,dongsheng,39.8161,109.9776,269.69,81,0,9.05,CN,1612475170
557,batticaloa,7.7102,81.6924,298.08,80,100,4.77,LK,1612475170
558,zhangye,38.9342,100.4517,270.20,73,0,2.76,CN,1612475103
559,carutapera,-1.1950,-46.0200,299.94,79,59,3.40,BR,1612475170


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# **Note:** if you having trouble displaying the maps,
# try running `jupyter nbextension enable --py gmaps` in your environment and retry.

# For max intensity in the heat map, try setting it to the highest humidity found in the data set.
gmaps.configure(api_key=g_key)
locations=vacation_df[['Lat', 'Lng']].astype(float)
humidity=vacation_df["Humidity"].astype(float)
fig=gmaps.figure()
heatmap=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Create a heat map that displays the humidity for every city from Part I.
fig.add_layer(heatmap)
fig # I don't know why my map looks multiplied!

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned
# by your API requests to a reasonable number.

lat_spots=vacation_df.loc[vacation_df['Lat']<=60]
lat_spots=lat_spots.dropna()
lat_spots # NOTE: be careful with the API here!

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,bowen,-20.0167,148.2333,298.15,88,81,3.09,AU,1612475095
2,hermanus,-34.4187,19.2345,293.15,78,10,0.45,ZA,1612475095
4,bredasdorp,-34.5322,20.0403,293.15,77,98,2.57,ZA,1612475095
5,padang,-0.9492,100.3543,298.15,94,40,0.51,ID,1612475095
6,el playon,7.4713,-73.2031,300.15,57,20,4.12,CO,1612475095
...,...,...,...,...,...,...,...,...,...
556,dongsheng,39.8161,109.9776,269.69,81,0,9.05,CN,1612475170
557,batticaloa,7.7102,81.6924,298.08,80,100,4.77,LK,1612475170
558,zhangye,38.9342,100.4517,270.20,73,0,2.76,CN,1612475103
559,carutapera,-1.1950,-46.0200,299.94,79,59,3.40,BR,1612475170


In [6]:
wind_spots=lat_spots.loc[lat_spots['Wind Speed']<=8]
wind_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,bowen,-20.0167,148.2333,298.15,88,81,3.09,AU,1612475095
2,hermanus,-34.4187,19.2345,293.15,78,10,0.45,ZA,1612475095
4,bredasdorp,-34.5322,20.0403,293.15,77,98,2.57,ZA,1612475095
5,padang,-0.9492,100.3543,298.15,94,40,0.51,ID,1612475095
6,el playon,7.4713,-73.2031,300.15,57,20,4.12,CO,1612475095
...,...,...,...,...,...,...,...,...,...
555,syracuse,37.0881,15.2792,289.82,77,0,5.14,IT,1612475170
557,batticaloa,7.7102,81.6924,298.08,80,100,4.77,LK,1612475170
558,zhangye,38.9342,100.4517,270.20,73,0,2.76,CN,1612475103
559,carutapera,-1.1950,-46.0200,299.94,79,59,3.40,BR,1612475170


In [7]:
humidity_spots=wind_spots.loc[(wind_spots['Humidity']<=50)&(wind_spots['Humidity']>=40)]
humidity_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,jamestown,42.0970,-79.2353,276.48,44,1,4.63,US,1612474950
25,san patricio,28.0170,-97.5169,303.71,48,20,5.66,US,1612475097
26,catamarca,-28.4696,-65.7852,302.59,50,11,0.45,AR,1612475097
31,laguna,38.4210,-121.4238,288.71,44,1,4.63,US,1612475098
78,coahuayana,18.7333,-103.6833,304.15,42,20,2.90,MX,1612475107
97,aswan,24.0934,32.9070,288.81,45,0,4.71,EG,1612475110
102,ancud,-41.8697,-73.8203,299.15,41,0,4.63,CL,1612475069
125,bethel,41.3712,-73.4140,277.15,48,1,2.57,US,1612475114
133,marco,25.9726,-81.7290,293.15,40,1,3.09,US,1612475116
134,wilmington,34.2257,-77.9447,283.15,40,1,3.09,US,1612475019


In [8]:
cloud_spots=humidity_spots.loc[humidity_spots['Cloudiness']<=15]
cloud_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,jamestown,42.0970,-79.2353,276.48,44,1,4.63,US,1612474950
26,catamarca,-28.4696,-65.7852,302.59,50,11,0.45,AR,1612475097
31,laguna,38.4210,-121.4238,288.71,44,1,4.63,US,1612475098
97,aswan,24.0934,32.9070,288.81,45,0,4.71,EG,1612475110
102,ancud,-41.8697,-73.8203,299.15,41,0,4.63,CL,1612475069
125,bethel,41.3712,-73.4140,277.15,48,1,2.57,US,1612475114
133,marco,25.9726,-81.7290,293.15,40,1,3.09,US,1612475116
134,wilmington,34.2257,-77.9447,283.15,40,1,3.09,US,1612475019
168,khandbari,27.3747,87.2039,283.66,46,0,1.16,NP,1612475120
176,parana,-31.7320,-60.5238,302.59,42,5,3.58,AR,1612475121


In [9]:
temperature_spots=cloud_spots.loc[(cloud_spots['Max Temp']<=300)&(cloud_spots['Max Temp']>-20)]
temperature_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,jamestown,42.0970,-79.2353,276.48,44,1,4.63,US,1612474950
31,laguna,38.4210,-121.4238,288.71,44,1,4.63,US,1612475098
97,aswan,24.0934,32.9070,288.81,45,0,4.71,EG,1612475110
102,ancud,-41.8697,-73.8203,299.15,41,0,4.63,CL,1612475069
125,bethel,41.3712,-73.4140,277.15,48,1,2.57,US,1612475114
133,marco,25.9726,-81.7290,293.15,40,1,3.09,US,1612475116
134,wilmington,34.2257,-77.9447,283.15,40,1,3.09,US,1612475019
168,khandbari,27.3747,87.2039,283.66,46,0,1.16,NP,1612475120
196,los llanos de aridane,28.6585,-17.9182,289.15,48,0,3.09,ES,1612475123
223,lompoc,34.6391,-120.4579,291.15,48,1,5.14,US,1612475126


In [10]:
country_spots=temperature_spots.loc[(temperature_spots['Country']!='US')&(temperature_spots['Country']!='MX')]
country_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,aswan,24.0934,32.9070,288.81,45,0,4.71,EG,1612475110
102,ancud,-41.8697,-73.8203,299.15,41,0,4.63,CL,1612475069
168,khandbari,27.3747,87.2039,283.66,46,0,1.16,NP,1612475120
196,los llanos de aridane,28.6585,-17.9182,289.15,48,0,3.09,ES,1612475123
360,miram shah,33.0086,70.0697,282.07,50,0,2.40,PK,1612475143
430,monywa,22.1167,95.1333,290.31,46,0,1.79,MM,1612475152
488,santa cruz de la palma,28.6835,-17.7642,289.15,48,0,3.09,ES,1612475161
504,gualeguay,-33.1416,-59.3097,297.65,47,0,2.62,AR,1612475163


In [11]:
perfect_spots=country_spots.loc[(country_spots['Lat']>=20)&(country_spots['Lat']<=30)]
perfect_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,aswan,24.0934,32.9070,288.81,45,0,4.71,EG,1612475110
168,khandbari,27.3747,87.2039,283.66,46,0,1.16,NP,1612475120
196,los llanos de aridane,28.6585,-17.9182,289.15,48,0,3.09,ES,1612475123
430,monywa,22.1167,95.1333,290.31,46,0,1.79,MM,1612475152
488,santa cruz de la palma,28.6835,-17.7642,289.15,48,0,3.09,ES,1612475161


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
hotel_df=perfect_spots

hotel_df['Hotel Name']=""
params={
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(base_url, params=params)
    response=response.json()
    print(json.dumps(response, indent=4, sort_keys=True))
    try:
        hotel_df.loc[index, "Hotel Name"]=response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Error.")
hotel_df.info() # I don't know why i'm getting a settingwithcopywarning

<ipython-input-12-6d6ffec31c44>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=""


{
    "html_attributions": [],
    "next_page_token": "ATtYBwI_OtMN2tSbvI3ptEyDVJhxM4ckFIv3PJ6CQIEaUrrduBjPDiTF0LUylvvIOvu9UEIJ_ZWaYuUXW0OahafuW_k15P0U2udE0gcO_R54weTfQvQ_OPv4lHHRU0Gd8Lx8C9mFkMpwuVrjuEYEuCrpH7gtOdNFPm2O1ymWomb0DtJmijTD3BiKE1edN8eys8LwOejCuRgPaj24hzopMah-b-MklFo1qeqSqPuaIr5w68bOt6MUXB6Wsxbk7yCqibu2ARZ2pcYSMT6f7y1m5rfHSY9gRskvCP3pNJBRct03dUeS1jLF-NLp5jrXmmpjfO4hAbUUtU6JkNTpt1yyzZItY8RmjWut9Nr5gyE1LVuz1mEUOMOgCsPUZb4rx7gIBoMWxmB8-1LcGi7qHmmm9pktj9ap-m0DxR1suzBohbKYWmTKZQfZawzUJptLg9SI9v-krg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 24.0823135,
                    "lng": 32.8873158
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.08366332989272,
                        "lng": 32.88866562989272
                    },
                    "southwest": {
                        "lat": 24.0809636

C:\Users\pamhi\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.3771842,
                    "lng": 87.20705869999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.37856102989272,
                        "lng": 87.20849567989272
                    },
                    "southwest": {
                        "lat": 27.37586137010727,
                        "lng": 87.20579602010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Barun Sankhuwasabha",
            "photos": [
                {
                    "height": 720,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/108835721849341073031\

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.1262073,
                    "lng": 95.12319939999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.12756722989272,
                        "lng": 95.12456172989273
                    },
                    "southwest": {
                        "lat": 22.12486757010728,
                        "lng": 95.12186207010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Win Unity Hotel",
            "photos": [
                {
                    "height": 2268,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/103909992476675012825\">SUTHIPO

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIls7A85kwM-8xqO_BNA4qPQfr2sCItxdhwA1DWhwNhYQOvXQS3osKikNRYW7pFyKbcWYJSm6i8jgCuh0B1-BYrnYs70va-jZDnIZM9PQcqJL4j7s44mTBeHH4Gwy4u2RGO2frMvaxHIB7ldhNJBicOVNCJGSZe09PRxZGMc-4CD8txVJRLvm0GloVN8q9vgMa7IfgodhZ0e6DzPbr4D35MXGttiXEA2pK9FTt2_sMoCfy5bnfVz8KlOk6qel0B8jx9p_1h8GJFrUU2p1ZWOfhWSyPavLf3bNiDyYuPK-wPpmHVH4xwdPk93IXXR3-p8r3JCXJ-E8LmRQIX7J5IAveA1q57L8J5cvu46chvFC-sBglOVxvolKAzV-aPXtA_IQtg8dGOPJINXEuICUa9QjyqC0iul_9iZ-D_nEFDzfV7Q3zk0ofBoxbf3jpRlx1kSg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 28.6494442,
                    "lng": -17.7580869
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.65072302989272,
                        "lng": -17.75694617010728
                    },
                    "southwest": {
                        "lat": 28.64802

In [13]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
97,aswan,24.0934,32.9070,288.81,45,0,4.71,EG,1612475110,Sofitel Legend Old Cataract Aswan
168,khandbari,27.3747,87.2039,283.66,46,0,1.16,NP,1612475120,Hotel Barun Sankhuwasabha
196,los llanos de aridane,28.6585,-17.9182,289.15,48,0,3.09,ES,1612475123,Hotel Hacienda de Abajo
430,monywa,22.1167,95.1333,290.31,46,0,1.79,MM,1612475152,Win Unity Hotel
488,santa cruz de la palma,28.6835,-17.7642,289.15,48,0,3.09,ES,1612475161,H10 Taburiente Playa


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map

# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.
fig_layout={
    'Width':'900px',
    'Height':'600px',
    'Border':'1px solid black',
    'Padding':'1px',
    'Margin':'0 auto 0 auto'
}
final_fig=gmaps.figure(layout=fig_layout,zoom_level=3,center=(30,40))

markers=gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)
final_fig.add_layer(heatmap)
final_fig.add_layer(markers)

# Display figure
final_fig
plt.savefig('output_data/vacation_spots.png')
final_fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [ ]:
#For Part II, you must include a screenshot of the heatmap you create and include it in your submission.